In [1]:
import tensorflow as tf
import numpy as np

/home/hojoon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/hojoon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-3
total_epoch = 30
batch_size  = 256

input_size  = 28
hidden_size = 128
class_size  = 10
step_size   = 28

### Build Graph

In [4]:
X = tf.placeholder(tf.float32, [None, step_size, input_size])
Y = tf.placeholder(tf.float32, [None, class_size])

In [5]:
W = tf.Variable(tf.random_normal([hidden_size, class_size]))
b = tf.Variable(tf.random_normal([class_size]))

In [6]:
"""
Creating RNN

1. Make Cell
2. Initialize State
3. Make RNN

""" 

cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
# initial_state = cell.zero_state(batch_size, dtype = tf.float32)  
outputs, states = tf.nn.dynamic_rnn(cell = cell, inputs = X, dtype = tf.float32) # if initial state is not designated, it will automatically assigned to be zero

# 'outputs' is a tensor of shape [batch_size, max_time, 256]

print(cell)
print(outputs, states)

Tensor("rnn/transpose:0", shape=(?, 28, 128), dtype=float32) Tensor("rnn/while/Exit_2:0", shape=(?, 128), dtype=float32)


In [7]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

print(outputs)

Tensor("strided_slice:0", shape=(?, 128), dtype=float32)


In [8]:
score = tf.matmul(outputs, W) + b

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op  = optimizer.minimize(loss)

In [9]:
# Prediction
is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.to_float(is_correct))

### Train Graph

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for epoch in range(total_epoch):
        total_loss = 0
        
        for _ in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape((batch_size, step_size, input_size))
            
            _, loss_val = sess.run([train_op, loss], feed_dict = {X:batch_xs, Y:batch_ys})
            
            total_loss += loss_val
    
        print('Epoch:','%04d' %(epoch+1),
              'Avg Loss: %.3f' %(total_loss / total_batch))

        
    #Prediction
    test_xs = mnist.test.images.reshape(-1, step_size, input_size)
    test_ys = mnist.test.labels
    
    print('Accuracy: ',sess.run(accuracy, feed_dict = {X:test_xs, Y:test_ys}))

Epoch: 0001 Avg Loss: 0.623
Epoch: 0002 Avg Loss: 0.272
Epoch: 0003 Avg Loss: 0.206
Epoch: 0004 Avg Loss: 0.169
Epoch: 0005 Avg Loss: 0.151
Epoch: 0006 Avg Loss: 0.131
Epoch: 0007 Avg Loss: 0.118
Epoch: 0008 Avg Loss: 0.116
Epoch: 0009 Avg Loss: 0.106
Epoch: 0010 Avg Loss: 0.096
Epoch: 0011 Avg Loss: 0.095
Epoch: 0012 Avg Loss: 0.090
Epoch: 0013 Avg Loss: 0.084
Epoch: 0014 Avg Loss: 0.081
Epoch: 0015 Avg Loss: 0.076
Epoch: 0016 Avg Loss: 0.075
Epoch: 0017 Avg Loss: 0.073
Epoch: 0018 Avg Loss: 0.073
Epoch: 0019 Avg Loss: 0.067
Epoch: 0020 Avg Loss: 0.066
Epoch: 0021 Avg Loss: 0.068
Epoch: 0022 Avg Loss: 0.059
Epoch: 0023 Avg Loss: 0.062
Epoch: 0024 Avg Loss: 0.062
Epoch: 0025 Avg Loss: 0.061
Epoch: 0026 Avg Loss: 0.054
Epoch: 0027 Avg Loss: 0.049
Epoch: 0028 Avg Loss: 0.057
Epoch: 0029 Avg Loss: 0.050
Epoch: 0030 Avg Loss: 0.048
Accuracy:  0.9736
